# inverse operation of polygonize

In [1]:
%pylab inline
import geopandas as gpd
import logging as log
import fiona, shapely
from shapely.geometry import LineString, MultiPolygon
from shapely.ops import polygonize
from slib import *
import geopandas
log.getLogger().setLevel(log.INFO)

mpl.rcParams['figure.figsize'] = [12, 12.0]



# here are a vector layer with points. each points need to have a field with an unique id for each geological unit
unit_uuid_name = "Unit_Code" # the name of the field to use to populate the map.
layer_name = "Apl_Strat_full"
layer_name = "subregion_fixed"
# layer_name = "cleaned"

# output data - geopackage
inputfname = "/data/SciDataHub/projects/map_validation/PM-MOO-MS-SPAApollo_01.gpkg"
# inputfname = "/home/luca/vertices_snapped.gpkg"

# dist = 1.0 # extend of 1 meters all the extreme segments of contacts - needed to ensure proper intersections


Populating the interactive namespace from numpy and matplotlib


In [2]:
startmap = geopandas.GeoDataFrame.from_file(inputfname,layer=layer_name)
startmap.to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="input")

In [3]:
t= 0.1
geom = startmap.geometry
sgeom = [g.simplify(t) for g in geom]
startmap.geometry = geom

startmap.to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="input_simplified")

In [4]:
from shapely.geometry import LineString
from shapely.ops import nearest_points


In [5]:
class EdgeWrapper:
    def __init__(self, point1, point2):
        assert(isinstance(point1, PointWrapper))
        assert(isinstance(point2, PointWrapper))
        self.p1 = point1
        self.p2 = point2
        self.updateGeom()
        self.enlisted_points = []

        
        
    def getVertices(self):
        return np.row_stack([self.p1.getPoint(), self.p2.getPoint()])
    
    def updateGeom(self):
        self._geom = LineString(self.getVertices())
    
    def asGeom(self):
        return self._geom
    
    def enlistPointForAddition(self, point):
        if point not in self.enlisted_points:
            self.enlisted_points.append(point)
            
    
        
    

    
        

class PointListWrapper:
    def __init__(self, line):
        self.geom = line
        self.points = np.array(line)
        self.pointset = []
        self.edgeset = []
        self.update()
        
    def updatePointSet(self):
        self.pointset = []
        for i, point in enumerate(self.points):
            self.pointset.append(PointWrapper(self, i))
            
    def updateEdgeSet(self):
        self.edgeset = []
        pts = self.pointset
        for p1, p2 in zip(pts, pts[1:] + [pts[0]]):
            self.edgeset.append(EdgeWrapper(p1, p2))
            
    def update(self):
        self.updatePointSet()
        self.updateEdgeSet()
        
        
    def setPoint(self, pid, point):
        self.points[pid] = point
                
            

class PointWrapper:
    def __init__(self, parent, pid):
        assert(isinstance(parent, PointListWrapper))
        self.id = pid
        self.parent = parent
        
    def getPoint(self):
        return self.parent.points[self.id]
    
    def setPoint(self, point):
        self.parent.setPoint(self.id, point) 
        
    def asGeom(self):
        return shapely.geometry.Point(self.getPoint())

class PolygonWrapper:
    def __init__(self, geom):
        assert(isinstance(geom, Polygon))
        self.geom = geom # keep a ref to the old geom
        self.holes = [PointListWrapper(a) for a in geom.interiors]
        self.boundary = PointListWrapper(geom.exterior)
        
    def getAllPoints(self):
        out = []
        out += self.boundary.pointset
        for a in self.holes:
            out += a.pointset
        return out
    
    
    def getAllEdges(self):
        out = []
        out += self.boundary.edgeset
        for a in self.holes:
            out += a.edgeset
        return out
    
    def hasHoles(self):
        return len(self.holes) != 0
    
    def rebuildGeometry(self):
        return Polygon(self.boundary.points, [a.points for a in self.holes])

class MultiPolygonWrapper:
    def __init__(self, geom):
        assert(isinstance(geom, MultiPolygon))
        self.geom = geom
        self.polygons = [PolygonWrapper(w) for w in geom]
        
    def getAllPoints(self):
        out = []
        for p in self.polygons:
            out += p.getAllPoints()
        return out
    
    def getAllEdges(self):
        out = []
        for p in self.polygons:
            out += p.getAllEdges()
        return out
    
    def rebuildGeometry(self):
        return MultiPolygon([p.rebuildGeometry() for p in self.polygons])
        
def wrap(geoms):
    out = []
    for g in geoms:
        if isinstance(g, Polygon):
            out.append(PolygonWrapper(g))
        elif isinstance(g, MultiPolygon):
            out.append(MultiPolygonWrapper(g))
        else:
            log.error("not implemented case")
            
    return out
            
    
    
        
wrapped = wrap(startmap.geometry)    

allpts = []
for w in wrapped:
    allpts+= w.getAllPoints()
    
alledges = []
for w in wrapped:
    alledges+= w.getAllEdges()


points = np.row_stack([p.getPoint() for p in allpts])


# EDGES

In [6]:
threshold=10
orphan_pts= np.arange(len(allpts))

In [7]:
edges = []
for e in alledges:
    e.updateGeom()
    g = e.asGeom()
    g.wrapper = e
    edges.append(g)

from shapely.strtree import STRtree
tree= STRtree(edges)

In [8]:
points_to_add = []
added_points = []
for orphanid in orphan_pts:
    pt = allpts[orphanid]
    qpt = pt.asGeom()
    
    candidates = tree.query(qpt)
    
    for c in candidates:
        di = c.distance(qpt)
        
        if di > threshold:
            continue
            
        if di < 1e-16: # machine precision
            continue
            
        if (c.wrapper.p1 == pt) or  (c.wrapper.p2 == pt):
            log.debug("point is in this edge")
            continue
            
        edgewrapper = c.wrapper
        addpoint = allpts[orphanid].getPoint()
        
        line =  edgewrapper._geom

        proj_pt = line.interpolate(line.project( Point(addpoint)))
        
        dd = proj_pt.distance(line)
        if dd > 1e-6:
            log.warning(f"problem distance is {dd}")
            
        # Now combine with interpolated point on line
        proj_pt = np.array(proj_pt)
        
        
#         proj_pt = np.array(nearest_points(edgewrapper._geom, Point(point))[0])
        added_points.append(proj_pt)
        
        points_to_add.append([edgewrapper, proj_pt])
       
        
        edgewrapper.enlistPointForAddition(pt.getPoint().tolist())
        

In [9]:
o = []


for edge in alledges:
    if len(edge.enlisted_points) > 0:
        distances = [edge.asGeom().project(Point(p)) for p in edge.enlisted_points]
        sorting_ids = np.argsort(distances)
        
        ordered = np.array(edge.enlisted_points)[sorting_ids]
        n = len(ordered)
        
        
        
        
        
        edge.p1.parent.points = np.insert(edge.p1.parent.points, edge.p2.id, ordered, axis=0)
        
        pointset = edge.p1.parent.pointset
        for pp in pointset[edge.p1.id+1:]:
            pp.id += n
        
        
        
        
        o.append(edge)

        
asGdf([ee.asGeom() for ee in o]).to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="interested_edges")   

In [10]:
newg = [p.rebuildGeometry() for p in wrapped]
startmap.geometry = newg

startmap.to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="after_fixing_edges")

# EDGES DONE

In [11]:
def remove_true_equals(points, idp1, idpts):
    p1 = points[idp1].getPoint()
    out = []
    for id2 in idpts:
        p2 = points[id2].getPoint()
        
        if np.linalg.norm(p1-p2) > 1e-16: # comparing to machine precision
            out.append(id2)
            
    return out

In [12]:
from scipy.spatial import cKDTree





tree = cKDTree(points)

equal_pts = []
orphan_pts = []

for id, point in enumerate(points):
    found = tree.query_ball_point(point, threshold)
    
    found.remove(id)
    
    if len(found) == 0:
        # the point seems to not have any correspondence, but might be placed on an edge of another polygon
        # keep it for later analysis
        orphan_pts.append(id)
        continue
    
#     found = remove_true_equals(allpts, id, found)
        
    if len(found) > 0:
        equal_pts.append(sort([id]+found).tolist())
        orphan_pts.append(id)
        

In [13]:
len(orphan_pts)

26225

In [14]:
equal_pts
snapped = []
snapped_n = []
# from scipy.spatial import cKDTree

# tree = cKDTree(pts)
tofix = np.unique(sort(equal_pts)).tolist()
for fix in equal_pts:
    pts = np.row_stack([p.getPoint() for p in [allpts[id] for id in fix]])
    avg = np.mean(pts, axis=0)
    for id in fix:
        allpts[id].setPoint(avg)
        
    snapped.append(avg)
    snapped_n.append(len(fix))
    

In [15]:


newg = [Point(p) for p in snapped]
numbers = np.array(snapped_n)

assert(len(newg) == len(numbers))

df = asGdf(newg)

df.geometry = newg
df["sizes"] = numbers

df.to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="point_snapping")

In [16]:
df

,geometry,sizes
0,POINT (187823.779 -47195.496),3
1,POINT (189418.615 -47518.895),2
2,POINT (190067.542 -48365.876),2
3,POINT (190425.046 -49853.460),2
4,POINT (191937.223 -50835.592),2
...,...,...
26028,POINT (98205.832 333200.279),2
26029,POINT (98746.818 334505.577),2
26030,POINT (98787.642 336101.124),2
26031,POINT (98892.954 337382.540),2


# working on edges snap

In [17]:
edges = []
for e in alledges:
    e.updateGeom()
    g = e.asGeom()
    g.wrapper = e
    edges.append(g)

from shapely.strtree import STRtree
tree= STRtree(edges)

In [18]:
points_to_add = []
added_points = []
for orphanid in orphan_pts:
    pt = allpts[orphanid]
    qpt = pt.asGeom()
    
    candidates = tree.query(qpt)
    
    for c in candidates:
        di = c.distance(qpt)
        
        if di > threshold:
            continue
            
        if di < 1e-16: # machine precision
            continue
            
        if (c.wrapper.p1 == pt) or  (c.wrapper.p2 == pt):
            log.debug("point is in this edge")
            continue
            
        edgewrapper = c.wrapper
        addpoint = allpts[orphanid].getPoint()
        
        line =  edgewrapper._geom

        proj_pt = line.interpolate(line.project( Point(addpoint)))
        
        dd = proj_pt.distance(line)
        if dd > 1e-6:
            log.warning(f"problem distance is {dd}")
            
        # Now combine with interpolated point on line
        proj_pt = np.array(proj_pt)
        
        
#         proj_pt = np.array(nearest_points(edgewrapper._geom, Point(point))[0])
        added_points.append(proj_pt)
        
        points_to_add.append([edgewrapper, proj_pt])
       
        
        edgewrapper.enlistPointForAddition(pt.getPoint().tolist())
        


In [19]:
# a = np.random.randn(10,2)
# print(a)

# b =np.eye(2)
# print(b)

# np.insert(a, 3, b, axis=0)

In [20]:
o = []


for edge in alledges:
    if len(edge.enlisted_points) > 0:
        distances = [edge.asGeom().project(Point(p)) for p in edge.enlisted_points]
        sorting_ids = np.argsort(distances)
        
        ordered = np.array(edge.enlisted_points)[sorting_ids]
        n = len(ordered)
        
        
        
        
        
        edge.p1.parent.points = np.insert(edge.p1.parent.points, edge.p2.id, ordered, axis=0)
        
        pointset = edge.p1.parent.pointset
        for pp in pointset[edge.p1.id+1:]:
            pp.id += n
        
        
        
        
        o.append(edge)

        
asGdf([ee.asGeom() for ee in o]).to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="interested_edges")        

In [21]:
# o = []
# p = []
# for e,a in points_to_add:
#     a = a.tolist()
#     if e not in o:
#         o.append(e)
#         p.append([a])
        
#     else:
#         id = o.index(e)
#         if a not in p[id]:
#             p[id].append(a)
        
# for edge, points in zip(o, p):
#     for po in points:
#         edge.insertPointOnLine(po)

In [22]:
# pots = [Point(po) for po in np.concatenate(p)]
# asGdf(pots).to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="to_be_added")

In [23]:
# asGdf([ee.asGeom() for ee in o]).to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="interested_edges")

In [24]:
# asGdf(np.concatenate(p).tolist())

In [25]:
# for edge, pt in points_to_add:
#     edge.insertPointOnLine(pt)

In [26]:
newg = [p.rebuildGeometry().simplify(t) for p in wrapped]
startmap.geometry = newg

startmap.to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="cleaned")

In [27]:
ppp = np.array(added_points)
pp = [Point(p) for p in ppp]

In [28]:
asGdf(pp).to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="added")

In [29]:
considered = []
for orphanid in orphan_pts:
    qpt = allpts[orphanid].asGeom()
    considered.append(qpt)
    

In [30]:
asGdf(considered).to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="considered")

In [31]:
from shapely.ops import unary_union

In [32]:
for g in startmap.geometry.buffer(0):
    if g.is_valid is False:
        log.warning("is fucking none")

In [33]:
asGdf([unary_union(startmap.geometry.buffer(0))]).to_file("/home/luca/vertices_snapped.gpkg", driver="GPKG", layer="singlepoly")